In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np
import time

### Load original Data, sort by timestamp and save it

In [2]:
# df = pd.concat((pd.read_csv('events_train.csv'), pd.read_csv('purchases_train.csv')), ignore_index=True)
# df.sort_values(by=['timestamp'], inplace=True)
# df.to_csv('unified_df.csv')

In [2]:
df = pd.read_csv('unified_df.csv')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def get_unique_counts(df, columns):
    print('Unique value counts per column:')
    for column in columns:
        print('{:<25}{:>}'.format('Unique ' + column + 's:', len(df[column].unique())))

In [5]:
get_unique_counts(df, ['event_type', 'customer_id', 'product_id', 'title', 'category_name'])

Unique value counts per column:
Unique event_types:      2
Unique customer_ids:     2211386
Unique product_ids:      123837
Unique titles:           71441
Unique category_names:   714


##### Extract gender, clothes types?

In [6]:
# df.category_name.unique()[:10]

In [7]:
# df.category_name.str.count('>')

In [8]:
# df.category_name.apply(lambda x: str(x).split('>')[0] if len(str(x).split('>')) > 0 else 'empty').value_counts()

In [9]:
# df.price.describe().apply(lambda x: format(x, 'f'))

In [10]:
# df_purchases.price.plot.hist(bins=30)

In [11]:
# df_purchases.price.plot.box()

## Filling missing data

In [3]:
def get_missing_count(df, columns):
    print('Missing counts per column:')
    for column in columns:
        print('{:<25}{:>}'.format('Missing ' + column + 's:', df[column].isna().sum()))

In [4]:
get_missing_count(df, ['customer_id', 'event_type', 'product_id', 'title', 'category_name', 'price'])

Missing counts per column:
Missing customer_ids:    0
Missing event_types:     0
Missing product_ids:     0
Missing titles:          1813
Missing category_names:  3435732
Missing prices:          44886


Fill missing titles based on equal product_ids.

In [5]:
df.loc[df.title.isnull(), 'title'] = df.groupby('product_id')['title'].transform('first')
get_missing_count(df, ['title'])

Missing counts per column:
Missing titles:          30


Fill missing category_names based on equal product_ids.

In [6]:
df.loc[df.category_name.isnull(), 'category_name'] = df.groupby('product_id')['category_name'].transform('first')
get_missing_count(df, ['category_name'])

Missing counts per column:
Missing category_names:  25034


Fill missing titles based on equal category_names.

In [7]:
df.loc[df.title.isnull(), 'title'] = df.groupby('category_name')['title'].transform('first')
get_missing_count(df, ['title'])

Missing counts per column:
Missing titles:          26


Fill missing category_names based on equal titles.

In [8]:
df.loc[df.category_name.isnull(), 'category_name'] = df.groupby('title')['category_name'].transform('first')
get_missing_count(df, ['category_name'])

Missing counts per column:
Missing category_names:  8583


Fill missing price based on equal titles and category_names

In [9]:
df.loc[df.price.isnull(), 'price'] = df.groupby('title')['price'].transform('first')
df.loc[df.price.isnull(), 'price'] = df.groupby('category_name')['price'].transform('first')
get_missing_count(df, ['price'])

Missing counts per column:
Missing prices:          0


#### Map categorical columns to their codes for sparse format

In [10]:
train_size = round(df.shape[0]*0.85)
print('Train len for 85-15% split:', train_size)

Train len for 85-15% split: 12582632


In [11]:
customer_id_categorical = df[:train_size]['customer_id'].astype("category")
product_id_categorical = df[:train_size]['product_id'].astype("category")

In [12]:
customer_id_lookup = customer_id_categorical.cat.codes
product_id_lookup = product_id_categorical.cat.codes

In [148]:
customer_map = dict((y, x) for x, y in enumerate(customer_id_categorical.cat.categories ))
item_map = dict((x, y) for x, y in enumerate(product_id_categorical.cat.categories ))

In [91]:
customer_id_categorical[:100]

0     204685f9-00f0-4a36-bab1-e0b74673c3b1
1     552445a5-f41d-4b25-b479-4e776d9bee61
2     fac01f48-a03a-41f2-acd5-b5f4e407d9c2
3     552445a5-f41d-4b25-b479-4e776d9bee61
4     bc59a341-040f-41ea-8f33-cc4a02c2d85d
                      ...                 
95    6c5b6da2-7b63-4019-a592-16a823024c9b
96    41679604-367f-47e9-909c-fcbdc1c21f49
97    5351b884-174e-4d54-96e6-9d9f1f4ab60d
98    41679604-367f-47e9-909c-fcbdc1c21f49
99    ba0004ec-7c09-4311-b8d8-da42bd077ab2
Name: customer_id, Length: 100, dtype: category
Categories (1882180, object): [0000130d-9652-4cfb-8c9c-1b1d97b988ab, 000014d9-5e0c-4a15-8697-20ac41f9a388, 000027e5-4dbd-4b9f-9a56-e4414f325358, 0000461a-6b42-456b-a47c-bc34749e62ce, ..., ffffe78e-28e6-467e-98b4-b6ebe83ca14b, fffff354-dd1f-451b-8c41-f1c039560280, fffffdc5-d457-470f-9939-6bb5df9593ba, fffffed2-4e10-445e-af87-f9b67c84438a]

In [22]:
cooDf.getrow(238211).todense().nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([  1541,   2016,  16588,  18503,  18850,  21539,  22470,  24365,
         38039,  41448,  54134,  54197,  55519,  56316,  68472,  76581,
         77169,  84074,  87682, 100887, 104989, 111645]))

In [94]:
customer_id_lookup

0            238211
1            626251
2           1843948
3            626251
4           1386000
             ...   
12582627    1243553
12582628    1787379
12582629    1154005
12582630    1460282
12582631    1132155
Length: 12582632, dtype: int32

In [16]:
customer_map['204685f9-00f0-4a36-bab1-e0b74673c3b1']

238211

In [191]:
test_df = df[train_size:]

In [24]:
df[df['customer_id'] == '204685f9-00f0-4a36-bab1-e0b74673c3b1'].product_id.unique()

array(['f130c542-d793-41d0-88f6-d1c52b4fe8f0',
       '752a7f86-3b60-42b4-945c-9d25a83a9632',
       'da087c55-9ccf-4914-a23e-dc2984582c54',
       '941e630e-93bc-4326-b57b-c2f2cb42edbc',
       'bdd146bd-eba9-4db0-98cf-2a35d8ea3dcd',
       '79df83fc-7c78-4417-89a9-6dd4784a2dda',
       '23dc13de-88ac-4124-837b-e4016a289d6f',
       'a588416e-597c-4ef9-8afa-f315411811a9',
       '2809a461-277e-40a1-afdb-9d5aead6dfbc',
       '2e7079b4-9773-49c4-966d-c876fc9f3919',
       'b5d190c7-32b2-4210-9af7-03d71ba81fc8',
       '28c1ba9b-96ed-41ee-b5ab-c05de22ed6b7',
       '34957903-0b34-49f7-a243-b6b3fc074fdf',
       '046fd932-24cd-45c7-8bc9-899af5204e89',
       '035a620f-9789-4be9-921a-2dfdc3dec371',
       '7805a29f-6076-4802-a5ed-687ae2680abe',
       '75046ab2-a978-44a4-afbc-5ae30ce7f449',
       'a6eb2400-c46b-4e1f-9ce0-aa469d95fc8b',
       '51d65dad-a8be-4553-8034-2c50c5ccb67b',
       '3072b612-5b82-4d74-8fa2-35d4de1256a8',
       'e2eeb1d4-87ab-4981-b553-14f62710f022',
       '591db

Clean space

In [24]:
del df

## Create occurence sparse matrix

In [17]:
cooDf = sparse.coo_matrix((np.ones(train_size), 
    (customer_id_lookup, product_id_lookup)))

In [26]:
cooDf.shape

(1882180, 118612)

#### Normalize occurences

In [27]:
cooDf.data = np.log1p(cooDf.data)

In [28]:
# sdf = pd.DataFrame.sparse.from_spmatrix(cooDf)

In [29]:
# sdf.shape

customer_id lookup example

In [30]:
# [i for i, x in enumerate(customer_id_lookup) if x == 279709]

Define neighbours model

In [31]:
neigh = NearestNeighbors(n_neighbors=5, n_jobs=-1)

In [32]:
neigh.fit(cooDf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [33]:
cooDf.getrow(0)

<1x118612 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Row format>

# Popularity

In [219]:
density = cooDf.getnnz(axis=0)

In [222]:
cooDf.shape

(1882180, 118612)

In [221]:
density.shape

(118612,)

In [251]:
cooDf.getrow(12).nonzero()[0].shape

(23,)

In [238]:
cooDf.count_nonzero()

8159464

In [237]:
density[:4]

array([ 10, 233,  10,  16])

In [ ]:
lilDf = cooDf.tolil().transpose()

In [253]:
lilDf

AttributeError: 'function' object has no attribute 'shape'

In [239]:
[i.nnz for i in cooDf.tolil()]

[28,
 2,
 1,
 4,
 3,
 1,
 3,
 2,
 5,
 1,
 1,
 27,
 23,
 5,
 1,
 3,
 1,
 1,
 14,
 22,
 2,
 1,
 13,
 1,
 5,
 1,
 1,
 4,
 7,
 34,
 6,
 3,
 1,
 11,
 2,
 2,
 1,
 1,
 1,
 13,
 3,
 3,
 1,
 1,
 2,
 4,
 1,
 7,
 9,
 3,
 1,
 2,
 3,
 2,
 1,
 1,
 4,
 1,
 2,
 23,
 2,
 10,
 6,
 1,
 2,
 2,
 1,
 46,
 5,
 6,
 20,
 2,
 1,
 4,
 8,
 1,
 2,
 46,
 1,
 1,
 1,
 12,
 1,
 3,
 1,
 1,
 3,
 12,
 2,
 8,
 1,
 4,
 3,
 2,
 14,
 2,
 1,
 2,
 1,
 1,
 5,
 2,
 3,
 23,
 2,
 5,
 1,
 32,
 5,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 7,
 2,
 1,
 4,
 2,
 20,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 16,
 1,
 1,
 1,
 1,
 7,
 1,
 2,
 12,
 1,
 32,
 1,
 9,
 1,
 1,
 28,
 5,
 2,
 1,
 1,
 2,
 5,
 1,
 5,
 1,
 1,
 1,
 3,
 6,
 2,
 4,
 1,
 1,
 3,
 2,
 5,
 1,
 2,
 4,
 1,
 3,
 1,
 1,
 2,
 4,
 7,
 8,
 1,
 6,
 1,
 10,
 3,
 2,
 6,
 1,
 3,
 30,
 4,
 2,
 1,
 3,
 2,
 1,
 2,
 8,
 1,
 2,
 17,
 12,
 9,
 2,
 1,
 1,
 2,
 1,
 20,
 1,
 1,
 1,
 2,
 16,
 1,
 1,
 1,
 3,
 1,
 1,
 4,
 2,
 1,
 2,
 8,
 1,
 3,
 1,
 2,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1

# Model

In [211]:
class KNN_model:
    def __init__(self, n_neighbors, metric, coo, customer_map, product_map):
        self.n_neighbors = n_neighbors
#         self.training_lookup...
        self.model = NearestNeighbors(n_neighbors, metric=metric, n_jobs=-1)
        self.coo = coo
        self.customer_map = customer_map
        self.product_map = product_map
        
    def fit(self):
        self.model.fit(self.coo)
        
    def translate_relevances_to_product_ids(self, relevances):
        return list(map(lambda x: self.product_map[x[0]], relevances))
    
    
    
    def predict_10(self, customer_id):
        if customer_id in self.customer_map:
            customer_index = self.customer_map[customer_id]
            customer_row = self.coo.getrow(customer_index)
            
            # get neighbors
            neighbors = self.model.kneighbors(customer_row, n_neighbors=self.n_neighbors)
            # sum their items based on distance
#             print('Neighbors:\n', neighbors)
            
            item_relevances = {}
            for dist, idx in zip(neighbors[0][0], neighbors[1][0]):
                
                nb_items = self.coo.getrow(idx).nonzero()[1]
                relevance = 1 - dist
#                 print('On dist:', relevance,'idx:', idx)
                for nb_item in nb_items:
                    item_relevances[nb_item] = item_relevances.get(nb_item, 0) + relevance
                    
            # remove already interacted with
            own_items = customer_row.nonzero()[1]
#             print('Own items:\n', own_items)
            for i in own_items:
                item_relevances.pop(i, None)
            
#             print('Relevances:\n', item_relevances)
            sorted_relevances = sorted(item_relevances.items(), key=lambda kv: kv[1], reverse=True)
#             print('Relevances:\n', sorted_relevances[:10])
            
            product_recommendations = self.translate_relevances_to_product_ids(sorted_relevances[:10])
#             print('Returning list:\n', product_recommendations)
            
            return product_recommendations
            # return top 10
        else:
            print('missing')
            return []

In [212]:
neigh = KNN_model(10, metric='cosine', coo=cooDf, customer_map=customer_map, product_map=item_map)

In [213]:
neigh.fit()

In [216]:
len(test_df[:1000].customer_id.unique())

378

In [217]:
first_N = test_df[:1000].customer_id.unique()
hits = 0
total = 0
for customer in first_N:
    nbs = neigh.predict_10(customer)
    checked_products = test_df[test_df.customer_id == customer].product_id.unique()
    shared = len(set(set(checked_products) & set(nbs)))
    hits += shared
    total += len(checked_products)
    print(
        len(nbs), 
        len(checked_products), 
        shared
    )

print('Total:', total)
print('Hits:', hits)

0 3 0
10 13 0
1 2 0
9 10 0
1 5 0
0 2 0
10 9 1
0 14 0
5 141 0
missing
0 1 0
0 1 0
0 43 0
2 1 0
0 1 0
3 8 0
1 16 1
0 1 0
10 1 0
6 5 0
0 8 0
missing
0 19 0
0 3 0
0 1 0
6 1 0
9 1 0
missing
0 3 0
3 3 1
missing
0 1 0
10 1 0
0 10 0
2 1 0
0 5 0
missing
0 17 0
10 7 3
10 4 0
0 6 0
0 1 0
10 2 0
missing
0 21 0
missing
0 1 0
3 1 0
missing
0 3 0
0 2 0
missing
0 2 0
0 1 0
0 1 0
5 17 0
10 2 1
0 6 0
6 26 0
2 1 0
7 5 0
10 21 0
0 2 0
missing
0 5 0
1 7 0
10 71 2
8 1 0
2 1 0
0 13 0
10 4 0
3 12 0
0 1 0
3 3 0
4 18 0
3 3 0
10 4 0
0 1 0
10 1 0
0 6 0
10 29 0
0 1 0
missing
0 2 0
6 3 0
10 18 0
0 2 0
missing
0 7 0
missing
0 2 0
missing
0 2 0
missing
0 4 0
missing
0 1 0
10 44 1
10 1 0
0 2 0
2 6 0
0 5 0
8 8 0
missing
0 2 0
3 2 0
missing
0 2 0
0 1 0
0 2 0
0 30 0
missing
0 1 0
10 1 0
10 1 0
0 6 0
missing
0 1 0
10 2 1
missing
0 2 0
missing
0 1 0
missing
0 1 0
0 4 0
8 4 0
missing
0 1 0
1 3 0
0 1 0
6 9 0
0 1 0
4 17 0
0 1 0
missing
0 5 0
missing
0 1 0
0 1 0
0 3 0
10 38 0
missing
0 1 0
10 8 0
3 1 0
10 1 0
missing
0 1 0
10 

In [218]:
df_train

NameError: name 'df_train' is not defined

In [202]:

customer = 'fe7e8e6b-4360-43bb-ac7a-f656588edcab'
nbs = neigh.predict_10(customer)
set(
    set(test_df[test_df.customer_id == customer].product_id.unique()) 
    & 
    set(nbs)
)

Neighbors:
 (array([[0.        , 0.12961172, 0.13835956, 0.14188367, 0.14719713,
        0.14719713, 0.14719713, 0.14719713, 0.14719713, 0.14719713]]), array([[1871135,  784714, 1880134,  753830, 1618976, 1671656, 1199743,
         727164, 1837562,   57520]]))
On dist: 1.0 idx: 1871135
On dist: 0.8703882797784893 idx: 784714
On dist: 0.8616404368553293 idx: 1880134
On dist: 0.8581163303210331 idx: 753830
On dist: 0.8528028654224417 idx: 1618976
On dist: 0.8528028654224417 idx: 1671656
On dist: 0.8528028654224417 idx: 1199743
On dist: 0.8528028654224417 idx: 727164
On dist: 0.8528028654224417 idx: 1837562
On dist: 0.8528028654224417 idx: 57520
Own items:
 [109320  95360  60752  60728   7846]
Relevances:
 {77758: 0.8581163303210331}
Relevances:
 [(77758, 0.8581163303210331)]
Returning list:
 ['a8234140-b703-4e07-af6a-2bda9d33cec0']


set()

In [199]:
set(
    set(test_df[test_df.customer_id == '0cd3376b-bdcf-4e64-8d1d-1d088653a678'].product_id.unique()) 
    & 
    set(['43b5122a-5637-48ac-a64e-70008673a915', '30b65a6b-bc2e-4973-9a49-7a2774c98f23', 
         '63188188-87f1-4825-af3d-7c77158e013c', '87d78242-2d34-47d6-892a-ee588a903bd8', 
         '5bbb2109-dcbf-403f-b903-378d0c552268', '08269305-ba47-4a72-9e87-d887c9e33139', 
         '0723ada3-2634-4c0d-92f7-6ac60bd56088', 'fbec2c54-8926-40e7-95ae-696cfe1d921a', 
         '4c32410b-9fe2-446e-9356-c914a08425a8', '97ac182f-92ac-4f7f-8db6-17bff9e7d7f1'
    ])
)

set()

In [200]:
test_df[:10]

,Unnamed: 0,customer_id,timestamp,event_type,product_id,title,category_name,price
12582632,12420536,50a770d4-c1e1-4d3e-93f9-904308cd4fb5,2019-08-24 03:47:51.406775 UTC,view_item,6ee076bc-663e-4408-9ada-1ef87f67a448,Jack and Jones Champ Long Sleeve Worker Shirt,Men>Men Clothing>Shirts>Denim shirts,56.41
12582633,12420537,0cd3376b-bdcf-4e64-8d1d-1d088653a678,2019-08-24 03:47:54.429229 UTC,view_item,7726858c-6ffa-4a5e-a436-6dd1e410c6d3,SoulCal Tie Panel Swimsuit Ladies,Women>Women's Clothing>Swimmsuits & bikins,12.46
12582634,12420538,fe7e8e6b-4360-43bb-ac7a-f656588edcab,2019-08-24 03:47:56.178494 UTC,view_item,5a636e0f-b77e-4018-9ffe-039010a933ab,Lee Cooper Dan Mens Trainers,Men>Men Footwear>Sneakers>Low sneakers,20.45
12582635,12420539,d085fb95-0024-42db-9994-2847ebb476c3,2019-08-24 03:47:57.128794 UTC,view_item,d24226fe-2172-4f99-81ef-c537c3ec18f2,Children's winter boots LOAP CHOSEE,Children>Children's Footwear>Winter shoes>Chil...,15.21
12582636,12420540,f23e972d-51d2-4cab-836b-b7fec26e32a7,2019-08-24 03:47:58.658026 UTC,view_item,d4a36cae-f01e-4690-9cba-3580a4b607c5,Top Secret LADY'S JACKET,Women>Women's Clothing>Jackets & Coats>Quilted...,44.23
12582637,12420541,99ecc992-7922-49f4-bf9c-56495215a058,2019-08-24 03:48:00.635403 UTC,view_item,9fc925fa-1f33-448f-9a27-3dec41203558,Trendyol Ellie Black Snake Women's Wedge-Heele...,Women>Women's Footwear>Court shoes,27.97
12582638,12420542,feacd8d0-6e1d-4edd-8511-4cf6e882bbef,2019-08-24 03:48:03.006286 UTC,view_item,e9f9e3e2-a5e6-4356-ab74-ef3a9f564953,Ombre Clothing Men's hoodie B675,Men>Men Clothing>Sweatshirts>Hoodies,11.54
12582639,12420543,9ce0c3ee-3842-4304-9592-c65eae7f7f0e,2019-08-24 03:48:03.030838 UTC,view_item,13b8a859-05aa-4a4e-9e08-d90802322d21,Top Secret LADY'S DRESS,Women>Women's Clothing>Dresses>Casual Dress,23.58
12582640,12420544,521df20a-7fff-418e-a517-a8ebb28ce274,2019-08-24 03:48:09.793491 UTC,view_item,274f8460-d7b7-434b-baf7-bbbb1685132b,Functional sweatshirt men ERCO FINER M,Men>Men Clothing>Sweatshirts>Functional sweats...,20.92
12582641,12420545,539399d5-59a8-463e-983d-524c4294493a,2019-08-24 03:48:10.241834 UTC,view_item,64a263f9-e6dc-47c3-8397-54dfbbddb3a6,Donnay Poly Tracksuit Mens,Sports>Football>Training Clothes>Football Trac...,31.47


# Validation

In [42]:
unique_customers = test_df.customer_id.unique()

In [46]:
unique_customers.shape

(482672,)

In [ ]:
def evaluate_score(model, customers, df, customer_map):
    total = 0
    for user in customers:
        if user not in customer_map:
            print('missing customer')
        else:
            customer_id = customer_map[user]
            unique_items_interactions = np.unique(df.groupby('customer_id')['product_id'].get_group(user).values)
            top_10_predictions = model.predict(user)

#### Notes:

##### 6.11.2019
- kategoriu rozparsovat na viacero stlpcov
- TF-IDF na title
- titlov je menej ako ID, casto je rovnaky nazov ale ine id, kedze sa mozu lisit vo farbe
- tam kde chybaju data sa pozerat, ci nahodou rovnaky produkt niekde inde v datach nema doplnenu price/category_name aby som doplnil chybajuce
- ak nemam aj tak najdene chybajuce hodnoty, skusat z rovnakej kategorie doplnit priemer a pod.
- povedal som ze sa chcem zamerat na hladiny cien, kategorie, TF-IDF titles
- on povedal aby som sa pozrel na vztahmi medzi kategoriami, tj ked niekto kupoval hento tak potom casto kupil toto
- data spojit, su rovnake ale proste prisli v rozdielnych suboroch

##### 20.11.2019
- lematizacia, vectorizer, vyratat TF-IDF a ratat podobnosti, da sa to dat do elasticu ???
- CF maticu pre produkty a userov, neudrziavat stlpce, ktore maju malo (1) views/buys + taktiez mozu na kazdy produkt byt pripojene vektory s nejakym kontextom, tj. TF-IDF title/category
- knn?
- ak nepojde - content-based TF-IDF nad titles/categories